In [1]:
import os

# Set CWD to MLAgentBench
new_working_directory = os.path.join(os.getcwd(), '..', '..', '..')
os.chdir(new_working_directory)
print("New Working Directory:", os.getcwd())

New Working Directory: c:\Users\kevihuang\OneDrive - Microsoft\Desktop\projects\MLAgentBench


In [2]:
from MLAgentBench_v2.agents.agent import Agent

from types import SimpleNamespace
from MLAgentBench_v2.environment import Environment

No module named 'helm'
Could not load CRFM API key crfm_api_key.txt.
[Errno 2] No such file or directory: 'claude_api_key.txt'
Could not load anthropic API key claude_api_key.txt.


In [3]:
# Instantiate an environment
args = SimpleNamespace(
    task='home-data-for-ml-course',
    task_type='kaggle',
    log_dir='logs/house-price-testing__gpt-3.5-turbo__react',
    work_dir='workspace',
    max_steps=50,
    max_time=18000,
    device=0,
    python='/home/user/micromamba/envs/autogpt/bin/python',
    interactive=False,
    resume=None,
    resume_step=0,
    agent_type='ReActAgent', # Just for instantiation -- doesn't actually do anything
    llm_name='gpt-3.5-turbo',
    fast_llm_name='gpt-3.5-turbo',
    edit_script_llm_name='gpt-3.5-turbo',
    # llm_name='gpt-4',
    # fast_llm_name='gpt-4',
    # edit_script_llm_name='gpt-4',
    edit_script_llm_max_tokens=4000,
    agent_max_steps=50,
    actions_remove_from_prompt=[],
    actions_add_to_prompt=[],
    no_retrieval=False,
    valid_format_entires=None,
    max_steps_in_context=3,
    max_observation_steps_in_context=3,
    max_retries=4,
    langchain_agent='zero-shot-react-description'
)

env = Environment(args)

Initializing environment...
args namespace(task='home-data-for-ml-course', task_type='kaggle', log_dir='logs/house-price-testing__gpt-3.5-turbo__react', work_dir='workspace', max_steps=50, max_time=18000, device=0, python='/home/user/micromamba/envs/autogpt/bin/python', interactive=False, resume=None, resume_step=0, agent_type='ReActAgent', llm_name='gpt-3.5-turbo', fast_llm_name='gpt-3.5-turbo', edit_script_llm_name='gpt-3.5-turbo', edit_script_llm_max_tokens=4000, agent_max_steps=50, actions_remove_from_prompt=[], actions_add_to_prompt=[], no_retrieval=False, valid_format_entires=None, max_steps_in_context=3, max_observation_steps_in_context=3, max_retries=4, langchain_agent='zero-shot-react-description')
Preparing task home-data-for-ml-course , of type:  kaggle


In [4]:
class ReActAgent(Agent):
    def __init__(self, env):
        super().__init__(env)

    def run(self):
        print("--- Starting to run ReAct Agent ---")
        with open("MLAgentBench_v2/agents/ReAct/prompts/system_prompt.txt", "r") as f:
            self.system_prompt = f.read()

        MAX_STEPS = 1 # DEBUG: REMOVE WHEN YOU DON'T WANT TO HARD-CODE
        for count in range(MAX_STEPS):
            # Create the prompt for function calling
            assert('action' in self.files_action_result_history[0].keys() and 'result' in self.files_action_result_history[0].keys() and 'files' in self.files_action_result_history[0].keys())
            formatted_answer_states = ""
            for idx, answer_state in enumerate(self.files_action_result_history):
                formatted_answer_states += "\nStep: " + str(idx) 
                formatted_answer_states += "\nFiles: " + str(answer_state['files']) 
                formatted_answer_states += "\nAction: " + answer_state['action'] 
                formatted_answer_states += "\nResult: " + answer_state['result'] 
            self.initial_prompt = f"""You are a helpful research assistant. Given a research problem, files, tools, and at most {self.max_states} of your most recent files, action, result, and answer, your goal is to choose and take the next best action and tool that you think could lead to a better answer and get you closer to solving the research problem. 

            Research Problem: {self.research_problem}
            Current Files: {self.files}
            Tools / functions: {list(self.available_actions.keys())}
            Most recent files, action, result, and answer states (oldest to newest):
            {formatted_answer_states}        
            """

            # Log initial prompt
            with open(self.main_log_path, "a", 1) as log_file:
                log_file.write(f"\nCalling Function Calling API with initial prompt: ")
                log_file.write(self.initial_prompt)
                log_file.write("\n")

            # FUNCTION CALLING AGENT: Call the function calling API by giving tools and available functions
            completion = self.complete_text_openai(system_prompt=self.system_prompt, user_prompt=self.initial_prompt, model=self.model, tools=self.tool_descriptions, available_functions=self.available_actions)

            # Log completion
            with open(self.main_log_path, "a", 1) as log_file:
                log_file.write(f"\Function Calling API output: ")
                log_file.write(completion)
                log_file.write("\n")

        return "Finished successfully! Final message: " + completion

agent = ReActAgent(env)

In [5]:
agent.run()

Starting to run Simple Function Calling Agent


--- LOGGING NEW ACTION ---
Step: 0
Calling function wrapped_complete_text_openai(args = (), kwargs = {'system_prompt': 'You are a helpful and first-rate research assistant.', 'user_prompt': "You are a helpful research assistant. Given a research problem, files, tools, and at most 32 of your most recent files, action, result, and answer, your goal is to choose and take the next best action and tool that you think could lead to a better answer and get you closer to solving the research problem. \n\n            Research Problem: Ask a home buyer to describe their dream house, and they probably won't begin with the height of the basement ceiling or the proximity to an east-west railroad. But this playground competition's dataset proves that much more influences price negotiations than the number of bedrooms or a white-picket fence.\n\nWith 79 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, this compe

'Finished successfully! Final message: Function calling complete! Action and result should be saved to history.'